In [2]:
!dir

import os
# Root directory of the project
ROOT_DIR = os.path.abspath("C:\\Users\\rmdu\\MaskTrack_RCNN")

 Volume in drive C is Windows
 Volume Serial Number is D626-90F1

 Directory of C:\Users\rmdu\MaskTrack_RCNN

06/14/2018  03:26 PM    <DIR>          .
06/14/2018  03:26 PM    <DIR>          ..
06/14/2018  02:20 PM               610 .gitignore
06/14/2018  03:24 PM    <DIR>          .idea
06/14/2018  11:46 AM    <DIR>          assets
06/14/2018  12:27 PM    <DIR>          dataset
06/14/2018  11:46 AM    <DIR>          images
06/14/2018  11:46 AM             1,118 LICENSE
06/14/2018  03:16 PM    <DIR>          logs
06/14/2018  11:46 AM                60 MANIFEST.in
06/14/2018  02:20 PM       257,557,808 mask_rcnn_coco.h5
06/14/2018  11:51 AM    <DIR>          mrcnn
06/14/2018  03:26 PM    <DIR>          project
06/14/2018  11:46 AM               108 README.md
06/14/2018  11:46 AM               130 requirements.txt
06/14/2018  11:46 AM    <DIR>          samples
06/14/2018  11:46 AM               102 setup.cfg
06/14/2018  11:46 AM             2,586 setup.py
               8 File(s)    257,5

In [3]:
import os
import re
from mrcnn import config, utils
import skimage.io
import numpy as np
from os.path import join, isfile

import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from project import wad_data


# TRAIN IMAGES ISSUE
# No. Files in video lists = 42,369
# No. Files in train_color = 39,222
# No. Files in train_label = 37,689



## Configuration

In [4]:


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib qt5 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


## Dataset

load_mask has been tested. green light .. green light

In [5]:
from project import wad_data

# Training dataset
dataset_train = wad_data.WADDataset()

dataset_train.load_data("C:\\Users\\rmdu\\MaskTrack_RCNN\\dataset\\wad", "train")

dataset_train.prepare()

dataset_val = wad_data.WADDataset()

dataset_val.load_data("C:\\Users\\rmdu\\MaskTrack_RCNN\\dataset\\wad", "val")

dataset_val.prepare()


In [6]:
print("Image Count: {}".format(len(dataset_val.image_ids)))
#print("Class Count: {}".format(dataset_train.num_classes))

Image Count: 50


In [7]:
import numpy as np
image_ids = np.random.choice(dataset_val.image_ids, 4)

#for image_id in image_ids:
image_id = image_ids[0]
image = dataset_val.load_image(image_id)
mask, class_ids = dataset_val.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_val.class_names)

In [12]:
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset_train.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset_train.class_names)

image_id  91 C:\Users\rmdu\MaskTrack_RCNN\dataset\wad\train_color\170908_061509080_Camera_5.jpg
image                    shape: (2710, 3384, 3)       min:    0.00000  max:  255.00000  uint8
mask                     shape: (2710, 3384, 9)       min:    0.00000  max:    1.00000  bool
class_ids                shape: (9,)                  min:    1.00000  max:    8.00000  int32
bbox                     shape: (9, 4)                min:    0.00000  max: 2809.00000  int32


## Testing

In [8]:
config = wad_data.WADConfig()

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\rmdu\MaskTrack_RCNN\logs\wad20180614T1529\mask_rcnn_wad_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mas

C:\Users\rmdu\Anaconda3\Lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


   1/1000 [..............................] - ETA: 36:03:44 - loss: 61.4070 - rpn_class_loss: 28.5932 - rpn_bbox_loss: 18.5560 - mrcnn_class_loss: 14.2578 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00

   2/1000 [..............................] - ETA: 34:32:44 - loss: 57.6885 - rpn_class_loss: 27.5910 - rpn_bbox_loss: 18.6480 - mrcnn_class_loss: 11.4495 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00